In [2]:
import pandas as pd
import scanpy as sc

import time

import requests
from bs4 import BeautifulSoup
import pandas as pd
from typing import List, Dict
from datetime import datetime
import os
import hashlib
import json
from pathlib import Path
import re

from io import BytesIO
from typing import Optional

import requests
from scipy import sparse
from io import BytesIO
from urllib import response

import pathlib
from multiprocessing import Pool
from tqdm import tqdm

In [3]:
_malignant_cell = "Malignant cells"
_cell_type = "cell_type"

In [4]:



content = requests.get("https://ngdc.cncb.ac.cn/cancerscem/downloads/CancerSCEM.metadata.xlsx").content

metadata = pd.read_excel(BytesIO(content))

In [5]:
metadata

,Project ID,Sample ID,Sample Type,Data source,BioProject,GEO,SRA,SRS,SRX,SRR,...,Project abstract,Construction protocol,Protocol,Instrument,Strategy,Library Layout,Cell count,Publication,PMID,DOI
0,MCC-001,MCC-001-01-1A,Tumour,NCBI (GEO),PRJNA483959,GSE117988,SRP155988,SRS3693909,SRX4579480,SRR7722938,...,Serial blood draws from a single patient and s...,"PBMC were isolated with standard FICOLL, tumor...",10X Genomics,Illumina HiSeq 2500,RNA-Seq,PAIRED,5477,"K.G. Paulson et al., Acquired cancer resistanc...",30250229,https://doi.org/10.1038/s41467-018-06300-3
1,MCC-001,MCC-001-02-1A,Tumour,NCBI (GEO),PRJNA483959,GSE117988,SRP155988,SRS3693908,SRX4579479,SRR7722937,...,Serial blood draws from a single patient and s...,"PBMC were isolated with standard FICOLL, tumor...",10X Genomics,Illumina HiSeq 2500,RNA-Seq,PAIRED,2401,"K.G. Paulson et al., Acquired cancer resistanc...",30250229,https://doi.org/10.1038/s41467-018-06300-3
2,MCC-001,MCC-001-03-1A,PBMC,NCBI (GEO),PRJNA483959,GSE117988,SRP155988,SRS3693910,SRX4579481,SRR7722939,...,Serial blood draws from a single patient and s...,"PBMC were isolated with standard FICOLL, tumor...",10X Genomics,Illumina HiSeq 2500,RNA-Seq,PAIRED,1991,"K.G. Paulson et al., Acquired cancer resistanc...",30250229,https://doi.org/10.1038/s41467-018-06300-3
3,MCC-001,MCC-001-04-1A,PBMC,NCBI (GEO),PRJNA483959,GSE117988,SRP155988,SRS3693911,SRX4579482,SRR7722940\t,...,Serial blood draws from a single patient and s...,"PBMC were isolated with standard FICOLL, tumor...",10X Genomics,Illumina HiSeq 2500,RNA-Seq,PAIRED,1064,"K.G. Paulson et al., Acquired cancer resistanc...",30250229,https://doi.org/10.1038/s41467-018-06300-3
4,MCC-001,MCC-001-05-1A,PBMC,NCBI (GEO),PRJNA483959,GSE117988,SRP155988,SRS3693912,SRX4579483,SRR7722941,...,Serial blood draws from a single patient and s...,"PBMC were isolated with standard FICOLL, tumor...",10X Genomics,Illumina HiSeq 2500,RNA-Seq,PAIRED,4524,"K.G. Paulson et al., Acquired cancer resistanc...",30250229,https://doi.org/10.1038/s41467-018-06300-3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1461,BRCA-134,BRCA-134-01-1A,Tumour,NCBI (GEO),PRJNA983305,GSE234832,SRP443229,SRS17968241,SRX20672578,SRR24911501; SRR24911502,...,Our analysis identifies intratumoral component...,Cellular suspensions were loaded on a Chromium...,10X Genomics,NextSeq 500,RNA-Seq,PAIRED,328,"Q. Song et al., Single-cell sequencing reveals...",37479733,https://doi.org/10.1038/s42003-023-05124-2
1462,BRCA-134,BRCA-134-02-1A,Tumour,NCBI (GEO),PRJNA983305,GSE234832,SRP443229,SRS17968242,SRX20672577,SRR24911503; SRR24911504,...,Our analysis identifies intratumoral component...,Cellular suspensions were loaded on a Chromium...,10X Genomics,NextSeq 500,RNA-Seq,PAIRED,654,"Q. Song et al., Single-cell sequencing reveals...",37479733,https://doi.org/10.1038/s42003-023-05124-2
1463,BRCA-134,BRCA-134-03-1A,Tumour,NCBI (GEO),PRJNA983305,GSE234832,SRP443229,SRS17968240,SRX20672576,SRR24911505; SRR24911506,...,Our analysis identifies intratumoral component...,Cellular suspensions were loaded on a Chromium...,10X Genomics,NextSeq 500,RNA-Seq,PAIRED,743,"Q. Song et al., Single-cell sequencing reveals...",37479733,https://doi.org/10.1038/s42003-023-05124-2
1464,BRCA-135,BRCA-135-01-1H,Tumour,NCBI (GEO),PRJNA780425,GSE188807,SRP346107,SRS11071870,SRX13138681,SRR16946746,...,To obtain more information about the lymph nod...,The concentration of single-cell suspension wa...,GEXSCOPE,Illumina NovaSeq 6000,RNA-Seq,PAIRED,2239,"K. Xu et al., Microenvironment components and ...",36148946,https://doi.org/10.3724/abbs.2022131


In [26]:
from io import BytesIO
from typing import Optional

import requests
from scipy import sparse


def get_dataframe(url: str, cache_dir: str = ".cache", compression: Optional[str] = None) -> Optional[pd.DataFrame]:
    # Create cache directory if it doesn't exist
    cache_path = Path(cache_dir)
    cache_path.mkdir(exist_ok=True)
    
    # Create a filename from URL using hash to avoid invalid characters
    url_hash = hashlib.md5(url.encode()).hexdigest()
    cache_file = cache_path / f"{url_hash}.parquet"
    
    # Check if file exists in cache
    if cache_file.exists():
        return pd.read_parquet(cache_file)
    
    # If not in cache, download and store
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        content = BytesIO(response.content)
        df = pd.read_csv(content, compression=compression, sep='\t', index_col=0)
        
        # Save to cache
        df.to_parquet(cache_file)
        return df
    
    return None


def get_anndata_from_sample_id(sample_id: str) -> Optional[sc.AnnData]:
    count_url = f'https://ngdc.cncb.ac.cn/cancerscem/downloads/expMatrixes/{sample_id}.counts.matrix.tsv.gz'
    celltype_url = f'https://ngdc.cncb.ac.cn/cancerscem/downloads/cell_annotation/{sample_id}.cell.type.txt'

    cell_type = get_dataframe(celltype_url)

    cell_type_counts = cell_type.value_counts()

    if not _malignant_cell in cell_type_counts.index:
        return 

    if cell_type_counts[_malignant_cell] < 50:
        return

    count_data = get_dataframe(count_url, compression="gzip")

    if (count_data is None) or (cell_type is None):
        return

    adata = sc.AnnData(count_data.transpose())
    adata.X = sparse.csr_matrix(adata.X)
    
    if sample_id.startswith("CRC-016-"):
        adata.obs_names = adata.obs_names.str.replace(".", "-")
        cell_type.index = cell_type.index.str.replace(".", "-")
    
    adata.obs = cell_type.loc[adata.obs_names].copy()
    adata.obs["sample_id"] = sample_id
    return adata



In [32]:


results_dir = pathlib.Path("data/CancerSCEM/")
results_dir.mkdir(exist_ok=True)
for project_id in tqdm(metadata["Project ID"].unique()):
    
    sample_ids =  metadata[metadata["Project ID"]==project_id]["Sample ID"].to_list()
    
    with Pool(processes=10) as pool:
        results = pool.map(get_anndata_from_sample_id, sample_ids)
    
    results = list(filter(lambda x: x, results))
    if not results: 
        continue
    adata = sc.concat(results)
    adata.obs_names = adata.obs_names + "_" +adata.obs["sample_id"].astype(str)

    adata.write_h5ad(results_dir.joinpath(f"{project_id.replace('-', '_')}.h5ad"))
    

  0%|          | 0/138 [00:00<?, ?it/s]/local/home/fbarkmann/envs/cancerfoundation/lib/python3.9/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
  1%|          | 1/138 [00:01<04:15,  1.87s/it]/local/home/fbarkmann/envs/cancerfoundation/lib/python3.9/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
  1%|▏         | 2/138 [00:04<05:18,  2.34s/it]/local/home/fbarkmann/envs/cancerfoundation/lib/python3.9/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
  2%|▏         | 3/138 [00:09<08:04,  3.59s/it]/local/home/fbarkmann/envs/cancerfoundation/lib/python3.9/site-packages/anndata/_core/anndata.py:1754: 

OSError: [Errno 28] Error writing bytes to file. Detail: [errno 28] No space left on device

In [31]:
results is None

False

In [25]:
sample_ids

['CRC-016-01-1A',
 'CRC-016-02-1A',
 'CRC-016-03-1A',
 'CRC-016-04-1A',
 'CRC-016-05-1A',
 'CRC-016-06-1A',
 'CRC-016-07-1A',
 'CRC-016-08-1A',
 'CRC-016-09-1A',
 'CRC-016-10-1A',
 'CRC-016-11-1A',
 'CRC-016-12-1A',
 'CRC-016-13-1A',
 'CRC-016-14-1A',
 'CRC-016-15-1A',
 'CRC-016-16-1A',
 'CRC-016-17-1A',
 'CRC-016-18-1A',
 'CRC-016-19-1A',
 'CRC-016-20-1A',
 'CRC-016-21-1A',
 'CRC-016-22-1A',
 'CRC-016-23-1A',
 'CRC-016-24-1A',
 'CRC-016-25-1A',
 'CRC-016-26-1A',
 'CRC-016-27-1A']